In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import math
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import os

import time
from joblib import Parallel, delayed

In [2]:
mod = SourceModule("""
    # include <stdio.h>
    
    #define N 1000
    
    
    __global__ void doIndexy(float *Z, int *indexx, int *indexy, int *P1a, int *P1b, float *P4, float *P3a, float *P3b, int totalSum, int totalThread )
    { 
        
        int idx = blockIdx.x * blockDim.x + threadIdx.x; 

        if(idx >= totalThread){
            return;
        }

        float sum_a = 0;
        float sum_b = 0;
        int j = indexy[idx]; 
        for (int i = 0; i < totalSum ; i++){
           if(Z[indexx[i]+j]>0.f){
               sum_a += 0.1*0.1*0.1 * Z[indexx[i]+j] * P1a[i];
               sum_b += 0.1*0.1*0.1 * Z[indexx[i]+j] * P1b[i];
           }
           
        }; 
        //printf("%d\t%f\t%f\t%d\t%d\t%d\t%d\\n",idx,sum1,Z[0],indexx[sizeof(indexy)/sizeof(indexy[0]) - 1], indexy[0],P1[0],total);
        P3a[idx] = sum_a*P4[idx]; 
        P3b[idx] = sum_b*P4[idx];
        
    }""")

In [8]:

# calculate equation 1
def eq1s2(t,T):
    a=np.empty(len(t[0]))
    for i in range(0,len(t[0])):
        arr = np.array([t[i,0],t[i,1],t[i,2],0])
        arr = np.sort(arr, axis=None)
        a[i]=1- np.heaviside (np.absolute(arr[3] -arr[1])- T, 0)
    return a

def eq1s4(t2,t3,t4,T):
    a=np.empty(len(t4))
    for i in range(0,len(t4)):
        arr = np.array([t2[i],t3[i],t4[i],0])
        arr = np.sort(arr, axis=None)
        a[i]= bool(np.heaviside (np.absolute(arr[2] -arr[1])- T, 0)) and bool((np.heaviside(np.absolute(arr[3] -arr[2])- T, 0)))
    return a

# for N=3 
def eq4(a,b,c,tau=1):
    ps = np.empty(len(a))
    for i in range(len(a)):
        if(b[i] >= a[i]) and (c[i]>=a[i]) and (c[i] >= b[i]):
            ps[i] = (1/tau**3)*(math.exp(-1*c[i]/tau))
        else: 
            ps[i] = 0
    return ps

In [2]:
def eq1(t,T):
    a=np.empty(t.shape[0])
    for i in range(0,len(t)):
        arr = t[i]
        arr = np.sort(arr, axis=None)
        count=2; j = 1; k=j+1;
        while k < arr.shape[0]:
            if (arr[k] - arr[j] >= T):
                count+=1; j+=1 ;k+=1;
            else:
                k+=1                
        a[i]= count
    return a

def eq4(a,d,tau=1):
    ps = np.empty(len(a))
    for i in range(len(a)):
        arr =a[i]
        if np.all(arr[:-1] <= arr[1:]):
            ps[i] = (1/tau**(d-1))*(math.exp(-1*arr[-1]/tau))
        else: 
            ps[i] = 0
    return ps

In [3]:
# parameter
tau = 1;
T = 1;
mu = 1.0;
lamda = 1.0;
bin_size = 0.10;
Pmin = 0;
Pmax = 1;
tmin = -1;
d=4

In [4]:
tmax= Pmax - tmin;
Pin_point= int((Pmax-Pmin)/bin_size +1);
obs_point=int((tmax-tmin)/bin_size + 1);
Zmax=tmax;
Zmin=tmin-Pmax;
Z_point=int((Zmax-Zmin)/bin_size + 1);

t=np.zeros((obs_point**3,d),dtype=float)
temp =np.arange(tmin,tmax+0.001,bin_size)
for i in range(d-1):
    temp1 = np.repeat(temp,obs_point**(d-i-2))
    temp2 = np.tile(temp1,obs_point**i)
    t[:,i] =temp2
del temp1,temp2
p1 = eq1(t,T)
del 

In [20]:
tauT=np.zeros((Pin_point**3,d-1),dtype=float)
temp =np.arange(Pmin,Pmax+0.001,bin_size)
for i in range(d-1):
    temp1 = np.repeat(temp,Pin_point**(d-i-2))
    temp2 = np.tile(temp1,Pin_point**i)
    tauT[:,i] =temp2
del temp1,temp2
p4 = eq4(tauT,d,tau)

In [28]:
d

4

In [48]:
for k1 in range (0,Pin_point):
    a=0
    for i in range(d-1):
        for j in range(0,i+1):
        
            a += Z_point**j*k
            print(a)

In [50]:
for i in range(d-1):
    for n, a in enumerate(list(range(Pin_point))):
        ans += 

SyntaxError: invalid syntax (<ipython-input-50-d7ba80e9740e>, line 2)

In [71]:
indexyM=np.zeros((Pin_point ** (d-2),d),dtype=int)

temp =np.array([i*Z_point for i in range(Pin_point)])
for i in range(d-1):
    temp1 = np.repeat(temp,Z_point**(d-i-2))
    temp2 = np.tile(temp1,Z_point**i)
    t[:,i] =temp2

ValueError: could not broadcast input array from shape (18491) into shape (29791)

In [67]:
temp

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.])

In [51]:
indexy = np.zeros(Pin_point ** (d-1),dtype=int)
for k in range (0,Pin_point):
    for k1 in range (0,Pin_point):
        a = [i for i in range( Z_point*Z_point*k + Z_point*k1 , Z_point*Z_point*k + Z_point*k1 + Pin_point)]
        print(a[0])
        indexy[ k*Pin_point*Pin_point + k1*Pin_point : k*Pin_point*Pin_point + (k1+1)*Pin_point] = a


0
41
82
123
164
205
246
287
328
369
410
1681
1722
1763
1804
1845
1886
1927
1968
2009
2050
2091
3362
3403
3444
3485
3526
3567
3608
3649
3690
3731
3772
5043
5084
5125
5166
5207
5248
5289
5330
5371
5412
5453
6724
6765
6806
6847
6888
6929
6970
7011
7052
7093
7134
8405
8446
8487
8528
8569
8610
8651
8692
8733
8774
8815
10086
10127
10168
10209
10250
10291
10332
10373
10414
10455
10496
11767
11808
11849
11890
11931
11972
12013
12054
12095
12136
12177
13448
13489
13530
13571
13612
13653
13694
13735
13776
13817
13858
15129
15170
15211
15252
15293
15334
15375
15416
15457
15498
15539
16810
16851
16892
16933
16974
17015
17056
17097
17138
17179
17220


In [45]:
indexy.shape[0]/Pin_point

121.0

In [5]:
start = time.time()
tmax= Pmax - tmin;
Pin_point= int((Pmax-Pmin)/bin_size +1);
obs_point=int((tmax-tmin)/bin_size + 1);
Zmax=tmax;
Zmin=tmin-Pmax;
Z_point=int((Zmax-Zmin)/bin_size + 1);

t4, t3,t2 = np.meshgrid(np.arange(tmin,tmax+0.001,bin_size), np.arange(tmin,tmax+0.001,bin_size), np.arange(tmin,tmax+0.001,bin_size));
t2 = t2.ravel()
t3 = t3.ravel()
t4 = t4.ravel()

tau4,tau3,tau2 = np.meshgrid(np.arange(Pmin,Pmax+0.001,bin_size), np.arange(Pmin,Pmax+0.001,bin_size), np.arange(Pmin,Pmax+0.001,bin_size));
tau2 = tau2.ravel()
tau3 = tau3.ravel()
tau4 = tau4.ravel()

p1s2=eq1s2(t3,t4,t2,T)
p1s4=eq1s4(t3,t4,t2,T)

f2 = pd.read_csv('savedist_4d.tsv',sep=' ', squeeze=True, header=None).values

# reading indexx and indexy
# indexy = np.load("indexy.npy")
# indexx = np.load("indexx.npy")



indexy = np.zeros(Pin_point ** (d-1),dtype=int)
for k in range (0,Pin_point):
    for k1 in range (0,Pin_point):
        a = [i for i in range( Z_point*Z_point*k + Z_point*k1 , Z_point*Z_point*k + Z_point*k1 + Pin_point)] 
        indexy[ k*Pin_point*Pin_point + k1*Pin_point : k*Pin_point*Pin_point + (k1+1)*Pin_point] = a


indexy = np.zeros(Pin_point ** (d-1),dtype=int)
for k in range (0,Pin_point):
    for k1 in range (0,Pin_point):
        a = [i for i in range( Z_point**Z_point*k + Z_point*k1 , Z_point*Z_point*k + Z_point*k1 + Pin_point)] 
        indexy[ k*Pin_point*Pin_point + k1*Pin_point : k*Pin_point*Pin_point + (k1+1)*Pin_point] = a

print("I am here")
indexy = indexy[::-1]


indexx = np.zeros(obs_point ** 3, dtype=int)

for k in range (0,obs_point): 
    for k1 in range (0,obs_point): 
        a = [i for i in range(Z_point*Z_point*k + k1*Z_point, Z_point*Z_point*k + k1*Z_point + obs_point )] 
        indexx[k*obs_point*obs_point + obs_point*k1 : k*obs_point*obs_point + (k1+1)*obs_point ] = a
#     np.save('indexy', indexy)
#     np.save('indexx', indexx)

print(indexx[0], indexy[0], indexx[-1], indexy[-1], f2.shape)

p4 = eq4(tau3,tau4,tau2,tau)

print("I am going into cuda")
# cuda

startC=time.time()
d_Z = cuda.mem_alloc(np.float32(f2).nbytes)
cuda.memcpy_htod(d_Z, np.float32(f2))

d_indexx = cuda.mem_alloc(np.int32(indexx).nbytes)
cuda.memcpy_htod(d_indexx, np.int32(indexx))

d_indexy = cuda.mem_alloc(np.int32(indexy).nbytes)
cuda.memcpy_htod(d_indexy, np.int32(indexy))

d_P1S2 = cuda.mem_alloc(np.int32(p1s2).nbytes) 
cuda.memcpy_htod(d_P1S2, np.int32(p1s2))

d_P1S4 = cuda.mem_alloc(np.int32(p1s4).nbytes) 
cuda.memcpy_htod(d_P1S4, np.int32(p1s4))

d_P4 = cuda.mem_alloc(np.float32(p4).nbytes) 
cuda.memcpy_htod(d_P4, np.float32(p4))

d_P3S2 = cuda.mem_alloc(np.float32(indexy).nbytes)
cuda.memcpy_htod(d_P3S2, np.float32(np.zeros_like(indexy)))
d_P3S4 = cuda.mem_alloc(np.float32(indexy).nbytes)
cuda.memcpy_htod(d_P3S4, np.float32(np.zeros_like(indexy))) 

func = mod.get_function("doIndexy")

blocksize = 128
gridsize = math.floor(len(indexy)/blocksize)
func(d_Z, d_indexx, d_indexy, d_P1S2, d_P1S4,d_P4, d_P3S2, d_P3S4, np.int32(len(p1s2)),np.int32(len(p4)), block=(blocksize,1,1), grid =(gridsize,1,1))

cuda.Context.synchronize()

h_test_outs2 = np.empty_like(np.float32(p4))
h_test_outs4 = np.empty_like(np.float32(p4))
cuda.memcpy_dtoh(h_test_outs2, d_P3S2)
cuda.memcpy_dtoh(h_test_outs4, d_P3S4)

cuda.Context.synchronize()

print("I am out of  cuda")

p=np.empty(2)
p[0] = bin_size**(d-1)*np.sum( h_test_outs2)
p[1] = bin_size**(d-1)*np.sum( h_test_outs4)

filename = "testfile" + str(mu) + str (lamda) +str(T) +str (tau)
file = open(filename,"w") 
file.write(str(p))
file.close() 
print("Prob: ", p, "Sec: ", time.time() - start, "inCuda ", time.time()- startC)

I am here
0 2030150 6090450 0 (8120601,)
I am going into cuda
I am out of  cuda
Prob:  [ 0.23490175  0.14351266] Sec:  55.73603916168213 inCuda  2.100482940673828
